In [50]:
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import os
import pandas as pd
import re
import datetime
from datetime import datetime

## Instalamos el driver de Google y entramos en - ElMundo-SALUD

In [28]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.elmundo.es/ciencia-y-salud.html')
sleep(1.5)

listaNoticias = driver.find_elements(By.XPATH, '//a[@class="ue-c-cover-content__link"]')
linkNoticias = list()

for i in range(20):
    link = listaNoticias[i].get_attribute('href')
    linkNoticias.append(link)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Pablo\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\Pablo\AppData\Local\Temp/ipykernel_7848/650325373.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [29]:
titulos = list()
entradillas = list()
textos = list()
fechas = list()

for i in linkNoticias: 
    try:
        driver.get(i)
        titulo = driver.find_element(By.XPATH, '//div[@class="ue-l-article__header-content"]').text
        titulos.append(titulo)
        entradilla = driver.find_element(By.CLASS_NAME, 'ue-c-article__standfirst').text
        entradillas.append(entradilla)
        texto = driver.find_element(By.XPATH, '//div[@class="ue-l-article__body ue-c-article__body"]').text
        textos.append(texto)
        fecha = driver.find_element(By.TAG_NAME, 'time').get_attribute('datetime')
        fechas.append(fecha)
        sleep(0.5)
    except:
        print("Error al recuperar la pagina", i)
        continue
        
df = pd.DataFrame(list(zip(titulos, entradillas, textos, fechas)), columns = ['Titulos','Entradillas', 'Textos', 'Fechas'])
df = df.assign(Noticiario= "ElMundo")
df["Fechas"] = df["Fechas"].replace(to_replace ='[T]\d*\:\d{2}\:\d{2}[Z]', value = ' ', regex = True)

Error al recuperar la pagina https://native.elmundo.es/2021/11/16nat/index.html#sec3
Error al recuperar la pagina https://www.elmundo.es/uestudio/2021/11/04/6183e4f6fc6c8384768b4665.html
Error al recuperar la pagina https://www.elmundo.es/uestudio/2021/11/11/618d34c9fc6c8399578b45d6.html
Error al recuperar la pagina https://www.elmundo.es/uestudio/2021/11/10/618b8f92fc6c83be5c8b4597.html


In [30]:
rutas = list()
n_Noticia = 0
n_dia = 0
for i in df.index:
    if(i!=0):
        if(df["Fechas"][i] == df["Fechas"][i-1]):
            n_dia+=1
        else:
            n_dia=0
    else:
        n_dia=0
    nombreFichero = "ElMundo/Salud/salud." + df["Fechas"][i] + ".{:03}".format(n_dia) + ".txt"
    nombreFichero = nombreFichero.replace(" ", "")
    rutas.append(nombreFichero)
    file = open(nombreFichero, "w")
    file.write("Titulo: " + df["Titulos"][i] + "\n\n")
    file.write("Entradilla: " + df["Entradillas"][i] + "\n\n")
    file.write(df["Textos"][i] + "\n\n")
    sleep(0.5)
    file.close()   
    n_Noticia += 1
df["rutas"] = rutas
df

,Titulos,Entradillas,Textos,Fechas,Noticiario,rutas
0,"SALUD\nEl macrobrote de Málaga, una voz de ala...",Después de los más de 80 contagiados tras una ...,Brote Los 68 profesionales infectados en el br...,2021-12-10,ElMundo,ElMundo/Salud/salud.2021-12-10.000.txt
1,COVID-19 Seguimiento de la pandemia\nLa incide...,Sanidad también notifica 84 muertes y 26.412 c...,Directo Coronavirus hoy\nEEUU La FDA da luz ve...,2021-12-09,ElMundo,ElMundo/Salud/salud.2021-12-09.000.txt
2,COVID\n¿Cómo han cambiado las vacunas el día a...,Es el tercer curso con el Covid como compañero...,"PREMIUM\nGRACIA PABLOS\nActualizado Jueves, 9 ...",2021-12-09,ElMundo,ElMundo/Salud/salud.2021-12-09.001.txt
3,CIENCIA Erupción en Cumbre Vieja\n¿Sin volcán ...,Los datos sugieren que podría parar en las pró...,"Ciencia 80 días a ciegas frente al volcán: ""Lo...",2021-12-10,ElMundo,ElMundo/Salud/salud.2021-12-10.000.txt
4,CERTIFICADO COVID En cada comunidad\nAsí se us...,Cada vez más comunidades autónomas recurren al...,PERE ÍÑIGO\nMadrid\nEL MUNDO GRÁFICOS\n@elmund...,2021-12-11,ElMundo,ElMundo/Salud/salud.2021-12-11.000.txt
5,"SALUD Julio García Comesaña, consejero de Sani...",Su región es la que mayor porcentaje de poblac...,PREMIUM\nLAURA G. IBAÑES\nMadrid\nCARMEN TORRE...,2021-12-11,ElMundo,ElMundo/Salud/salud.2021-12-11.001.txt
6,ACCIONES ANTE LA CRISIS CLIMÁTICA (Y VII)\nJav...,"Desde Alpedrete, armado con un móvil y con ""re...",Antonio Abellán El agitador de la huerta murci...,2021-12-10,ElMundo,ElMundo/Salud/salud.2021-12-10.000.txt
7,SALUD\nLa Agencia Europea del Medicamento dice...,"El organismo europeo también señala que, aunqu...",Directo Últimas noticias del coronavirus\nVacu...,2021-12-09,ElMundo,ElMundo/Salud/salud.2021-12-09.000.txt
8,ENFERMERA SATURADA Consultorio\nTodo lo que de...,Los glóbulos rojos son necesarios para transpo...,"Pandemia Síntomas de Ómicron: dolor muscular, ...",2021-12-12,ElMundo,ElMundo/Salud/salud.2021-12-12.000.txt
9,ASTRONOMÍA Crónicas del Cosmos\nLa pareja más ...,Hallan una pareja de agujeros negros supermasi...,Astronomía Una superburbuja en la Gran Nube de...,2021-12-11,ElMundo,ElMundo/Salud/salud.2021-12-11.000.txt


In [31]:
df.to_csv("rutas.csv")

## ElMundo-Tecnología

In [32]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.elmundo.es/tecnologia.html')
sleep(1.5)

listaNoticiasTech = driver.find_elements(By.XPATH, '//a[@class="ue-c-cover-content__link"]')
linkNoticiasTech = list()

for i in range(20):
    linkTech = listaNoticiasTech[i].get_attribute('href')
    linkNoticiasTech.append(linkTech)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Pablo\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\Pablo\AppData\Local\Temp/ipykernel_7848/2440164247.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [33]:
titulosTech = list()
entradillasTech = list()
textosTech = list()
fechasTech = list()

for i in linkNoticiasTech:  
    try:
        driver.get(i)
        titulo = driver.find_element(By.XPATH, '//div[@class="ue-l-article__header-content"]').text
        titulosTech.append(titulo)
        entradilla = driver.find_element(By.CLASS_NAME, 'ue-c-article__standfirst').text
        entradillasTech.append(entradilla)
        texto = driver.find_element(By.XPATH, '//div[@class="ue-l-article__body ue-c-article__body"]').text
        textosTech.append(texto)
        fecha = driver.find_element(By.TAG_NAME, 'time').get_attribute('datetime')
        fechasTech.append(fecha)     
        sleep(0.5)
    except:
        print("Error al recuperar la pagina", i)
        continue
        
df2 = pd.DataFrame(list(zip(titulosTech, entradillasTech, textosTech, fechasTech)), columns = ['Titulos','Entradillas', 'Textos', 'Fechas'])
df2 = df2.assign(Noticiario= "ElMundo")
df2["Fechas"] = df2["Fechas"].replace(to_replace ='[T]\d*\:\d{2}\:\d{2}[Z]', value = ' ', regex = True)

In [ ]:
rutas = list()
n_Noticia = 0
n_dia = 0
for i in df2.index:
    if(i!=0):
        if(df2["Fechas"][i] == df2["Fechas"][i-1]):
            n_dia+=1
        else:
            n_dia=0
    else:
        n_dia=0
    nombreFichero = "ElMundo/Tecnologia/tecnologia." + df2["Fechas"][i] + ".{:03}".format(n_dia) + ".txt"
    nombreFichero = nombreFichero.replace(" ", "")
    rutas.append(nombreFichero)
    file = open(nombreFichero, "w")
    file.write("Titulo: " + df2["Titulos"][i] + "\n\n")
    file.write("Entradilla: " + df2["Entradillas"][i] + "\n\n")
    file.write(df2["Textos"][i] + "\n\n")
    sleep(0.5)
    file.close()  
    n_Noticia += 1
df2["rutas"] = rutas
df = df.append(df2, ignore_index=True)
df

In [35]:
df.to_csv("rutas.csv")

## ELMundo-Ciencia (pendiente)

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.elmundo.es/ciencia-y-salud/ciencia.html')
sleep(1.5)

listaNoticiasSci = driver.find_elements(By.XPATH, '//a[@class="ue-c-cover-content__link"]')

linkNoticiasSci = list()

for noticia in listaNoticiasSci:
    linkSci = noticia.get_attribute('href')
    linkNoticiasSci.append(linkSci)


In [ ]:
titulosSci = list()
entradillasSci = list()
textosSci = list()
fechasSci = list()

for i in linkNoticiasSci:
    
    try:
        driver.get(i)
        titulo = driver.find_element(By.XPATH, '//div[@class="ue-l-article__header-content"]').text
        titulosSci.append(titulo)
        entradilla = driver.find_element(By.CLASS_NAME, 'ue-c-article__standfirst').text
        entradillasSci.append(entradilla)
        texto = driver.find_element(By.XPATH, '//div[@class="ue-l-article__body ue-c-article__body"]').text
        textosSci.append(texto)
        fecha = driver.find_element(By.TAG_NAME, 'time').get_attribute('datetime')
        fechasSci.append(fecha)
        
        sleep(0.5)
    except:
        continue


In [ ]:
df = pd.DataFrame(list(zip(titulosSci, entradillasSci, textosSci, fechasSci)), columns = ['Titulos','Entradillas', 'Textos', 'Fechas'])
df=df.assign(Noticiario= "ElMundo")

df["Fechas"] = df["Fechas"].replace(to_replace ='[T]\d*\:\d{2}\:\d{2}[Z]', value = ' ', regex = True)
df

In [ ]:
n_Noticia = 0
for i in df.index:
    nombreFichero = "Ciencia_"+ df["Fechas"][i] + df["Noticiario"][i] + str(n_Noticia)
    file = open(nombreFichero, "w")
    file.write(df["Titulos"][i]+ "\n")
    file.write("############################################ \n")
    file.write(df["Entradillas"][i]+ "\n")
    file.write("############################################ \n")
    file.write(df["Textos"][i]+ "\n")
    file.write("############################################ \n")
    file.write(df["Fechas"][i]+ "\n")
    sleep(0.5)
    file.close()
    
    n_Noticia += 1


## WebScraping ElPais-Sanidad (pendiente)

In [71]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://elpais.com/noticias/sanidad/')
sleep(1.5)

listaNoticiasSanidad_P = driver.find_elements(By.XPATH, '//a[@class="c_m_c _pr _db"]')
linkNoticiasSanidad_P = list()

for i in range(20):    
    linkSanidad_P = listaNoticiasSanidad_P[i].get_attribute('href')
    linkNoticiasSanidad_P.append(linkSanidad_P)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Pablo\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\Pablo\AppData\Local\Temp/ipykernel_7848/67765831.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
titulosSanidad_P = list()
entradillasSanidad_P = list()
textosSanidad_P = list()
fechasSanidad_P = list()
cont = 0

for i in linkNoticiasSanidad_P:    
    try:
        driver.get(i)
        titulo = driver.find_element(By.XPATH, '//h1[@class="a_t"]').text
        titulosSanidad_P.append(titulo)
        entradilla = driver.find_element(By.XPATH, '//h2[@class="a_st"]').text
        entradillasSanidad_P.append(entradilla)
        texto = driver.find_element(By.XPATH, '//div[@class="a_c clearfix"]').text
        textosSanidad_P.append(texto)
        fecha = driver.find_element(By.ID, 'article_date_p').get_attribute('data-date')
        fechasSanidad_P.append(fecha[0:10])
        print("titulo =", titulo)
        print("entradilla =", entradilla)
        print("texto =", texto)
        print("fechasSanidad_P =", fechasSanidad_P[cont])
        sleep(0.5)
        cont+=1
    except:
        print("Error al recuperar la pagina", i)
        continue
        
df2 = pd.DataFrame(list(zip(titulosSanidad_P, entradillasSanidad_P, textosSanidad_P, fechasSanidad_P)), columns = ['Titulos','Entradillas', 'Textos', 'Fechas'])
df2 = df2.assign(Noticiario= "ElPais")
df2

In [ ]:
rutas = list()
n_Noticia = 0
n_dia = 0
for i in df2.index:
    if(i!=0):
        if(df2["Fechas"][i] == df2["Fechas"][i-1]):
            n_dia+=1
        else:
            n_dia=0
    else:
        n_dia=0
    nombreFichero = "ElPais/Salud/salud." + df2["Fechas"][i] + ".{:03}".format(n_dia) + ".txt"
    nombreFichero = nombreFichero.replace(" ", "")
    rutas.append(nombreFichero)
    file = open(nombreFichero, "w")
    file.write("Titulo: " + df2["Titulos"][i] + "\n\n")
    file.write("Entradilla: " + df2["Entradillas"][i] + "\n\n")
    file.write(df2["Textos"][i] + "\n\n")
    sleep(0.5)
    file.close()  
    n_Noticia += 1
df2["rutas"] = rutas
df = df.append(df2, ignore_index=True)
df

In [ ]:
df.to_csv("rutas.csv")

## Salud 20minutos (pendiente)

In [78]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.20minutos.es/salud/')
sleep(1.5)

listaNoticiasSalud_20m = driver.find_elements(By.XPATH, '//div[@class="media-content"]')
linkNoticiasSalud_20m = list()

for noticia in listaNoticiasSalud_20m[0:20]:
    bLink = noticia.find_element(By.TAG_NAME, 'a')
    linkSalud_20m = bLink.get_attribute('href')
    linkNoticiasSalud_20m.append(linkSalud_20m)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Pablo\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
C:\Users\Pablo\AppData\Local\Temp/ipykernel_7848/49760380.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [80]:
titulosSalud_20m = list()
entradillasSalud_20m = list()
textosSalud_20m = list()
fechasSalud_20m = list()

for i in linkNoticiasSalud_20m:  
    try:
        driver.get(i)
        titulo = driver.find_element(By.XPATH, '//h1[@class="article-title "]').text
        titulosSalud_20m.append(titulo)
        entradilla = driver.find_element(By.XPATHTH, '//div[@class="article-intro "]').text
        entradillasSalud_20m.append(entradilla)
        texto = driver.find_elements(By.XPATH, '//p[@class="paragraph "]').text
        textosSalud_20m.append(texto)
        fecha = driver.find_element(By.XPATH, 'span[@class="article-date"]').text
        fechasSalud_20m.append(fecha)
        sleep(0.5)
    except:
        print("Error al recuperar la pagina", i)
        continue
        
df2 = pd.DataFrame(list(zip(titulosSalud_20m, entradillasSalud_20m, textosSalud_20m, fechasSalud_20m)), columns = ['Titulos','Entradillas', 'Textos', 'Fechas'])
df2 = df2.assign(Noticiario= "20minutos")
df2

Error al recuperar la pagina https://www.20minutos.es/videos/madrid/4922750-cuidados-paliativos-no-alargaron-vida-mi-hijo-si-la-ensancharon/
Error al recuperar la pagina https://www.20minutos.es/salud/cuando-hacer-test-antigenos-fiarnos-resultado-4922648/
Error al recuperar la pagina https://www.20minutos.es/salud/actualidad/bacteria-helicobapter-pylori-resistencia-antibioticos-4921058/
Error al recuperar la pagina https://www.20minutos.es/salud/sintoma-variante-omicron-manifiesta-por-noches-4922726/
Error al recuperar la pagina https://www.20minutos.es/salud/sintoma-puede-alertar-cancer-trago-cerveza-4921366/
Error al recuperar la pagina https://www.20minutos.es/videos/salud/el-cirujano-gallego-que-triunfa-en-123-paises-con-su-tecnica-para-el-cancer-de-pulmon-4913358/
Error al recuperar la pagina https://www.20minutos.es/videos/salud/los-nuevos-precios-de-los-medicamentos-cuales-suben-y-cuales-bajan-4912623/
Error al recuperar la pagina https://www.20minutos.es/videos/salud/estos-son-

,Titulos,Entradillas,Textos,Fechas,Noticiario
